In [1]:
import json
import datetime

In [2]:

def get_locations(filename):
    with open(filename) as f:
        return json.load(f)['locations']

loc_file = '\home\hlebaron98\google_location_data\google_location_data_at_2020-09\Takeout\Location History\Location History.json'

location_entries = get_locations(loc_file)



In [3]:
def is_within_location_range(loc_lat, loc_long, latitude_e7, longitude_e7, epsilon):
    return abs(loc_lat - latitude_e7) <= epsilon and abs(loc_long - longitude_e7) <= epsilon

In [4]:
TEST_EPSILON = 1000

start_range = 0
end_range = 999999999999999999999
desired_latitude = 407726160
desired_longitude = -1119124320
for i in range(len(location_entries)):
    if int(location_entries[i]['timestampMs']) >= end_range:
        break
    if int(location_entries[i]['timestampMs']) >= start_range:
       if is_within_location_range(desired_latitude, desired_longitude,
        location_entries[i]['latitudeE7'],location_entries[i]['longitudeE7'], TEST_EPSILON):
            print("found one: " + location_entries[i]['timestampMs'])

# 
# PROBLEM: How much time did you spend at a specific coordinate within EPSILON units of distance
# given an interval of time
#
#
   

    

found one: 1541394537000
found one: 1541542176000
found one: 1541542193957
found one: 1541542216223
found one: 1541645179480
found one: 1541645239562
found one: 1541645296514
found one: 1541645424507
found one: 1541711965000
found one: 1541711986000
found one: 1541712005767
found one: 1541712110000
found one: 1541712170000
found one: 1541800178000
found one: 1541819406000
found one: 1541871903000
found one: 1541871920093
found one: 1541871940202
found one: 1541878083000
found one: 1541907364000
found one: 1541956567000
found one: 1541956582000
found one: 1541956586655
found one: 1541956589000
found one: 1541956590000
found one: 1542158995068
found one: 1542330106000
found one: 1542475132000
found one: 1542492431000
found one: 1542492491000
found one: 1542500780000
found one: 1542510474000
found one: 1542727012000
found one: 1543083184000
found one: 1543085888000
found one: 1543085910000
found one: 1543100727000
found one: 1543124952000
found one: 1543129466000
found one: 1543144983000
